<a href="https://www.kaggle.com/code/syedfuzail1/google-agentic-course-project?scriptVersionId=281132449" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [88]:
!pip install -q google-genai
!pip install -q google-adk --no-deps


In [89]:
from google.adk.agents import Agent
from google import genai

print("ADK and genai imported successfully!")


ADK and genai imported successfully!


In [90]:
print("Importing required libraries...")

import os
import asyncio

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, load_memory
from google.adk.tools.agent_tool import AgentTool
from google.genai.types import Content, Part

print("✔ Libraries imported successfully!")

# Basic app constants
APP_NAME = "conceptloop_micro_tutor"
USER_ID = "demo_user"
SESSION_ID = "session_1"

# ==========================
# API KEY CONFIGURATION
# ==========================
API_KEY = "paste the api key here "   # 👈 Replace ONLY for testing
os.environ["GOOGLE_API_KEY"] = API_KEY
print("✔ GOOGLE_API_KEY loaded successfully!")

# Model
MODEL = "gemini-2.5-flash"
print("✔ Model set to:", MODEL)


Importing required libraries...
✔ Libraries imported successfully!
✔ GOOGLE_API_KEY loaded successfully!
✔ Model set to: gemini-2.5-flash


In [91]:
print("Initializing session and memory services...")

session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

print("✔ Session & Memory Services initialized successfully!")


Initializing session and memory services...
✔ Session & Memory Services initialized successfully!


In [92]:
print("Defining custom tool and sub-agents...")

def update_progress(topic: str, step_name: str, result: str) -> str:
    return f"Progress logged for topic='{topic}', step='{step_name}', result='{result}'"

planner_agent = Agent(
    name="planner_agent",
    model=MODEL,
    description="Breaks a topic into learning steps.",
    instruction=(
        "Break the given topic into 3–5 clear steps. "
        "Format: Step <number>: <short title> - <one sentence goal>"
    )
)

tutor_agent = Agent(
    name="tutor_agent",
    model=MODEL,
    description="Explains topic steps simply.",
    instruction=(
        "Explain the given step in extremely simple words using small examples."
    ),
    tools=[google_search],
)

quiz_agent = Agent(
    name="quiz_agent",
    model=MODEL,
    description="Creates mini quiz questions.",
    instruction=(
        "Ask 1-2 small questions and check the answer."
    ),
)

print("✔ Sub-agents and tools created!")


Defining custom tool and sub-agents...
✔ Sub-agents and tools created!


In [93]:
print("Wrapping sub-agents as tools and defining coach agent...")

planner_tool = AgentTool(agent=planner_agent)
tutor_tool = AgentTool(agent=tutor_agent)
quiz_tool = AgentTool(agent=quiz_agent)

coach_agent = Agent(
    name="coach_agent",
    model=MODEL,
    description="Coordinates planner, tutor, and quiz flow.",
    instruction=(
        "You are a study coach. Use planner → tutor → quiz flow. "
        "If user fails quiz, explain again and retry once. "
        "Log progress after each step."
    ),
    tools=[planner_tool, tutor_tool, quiz_tool, load_memory, update_progress],
)

print("✔ Coach agent created successfully!")


Wrapping sub-agents as tools and defining coach agent...
✔ Coach agent created successfully!


In [94]:
print("Setting up runner and execution flow...")

async def run_once(user_message: str):
    print(f"\n🚀 Starting new learning session for: '{user_message}'")

    # delete existing session first if present
    try:
        await session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
        print("🗑 Previous session deleted.")
    except:
        print("ℹ No previous session found, creating new session.")

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={"current_topic": user_message}
    )
    print("✨ New session created.")

    runner = Runner(
        agent=coach_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service,
    )
    print("✔ Runner initialized")

    content = Content(parts=[Part(text=user_message)], role="user")

    final_text = ""
    print("🤖 Running agent interaction...")

    # ---------------------------
    # UPDATED DEBUG EVENT VIEW
    # ---------------------------
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content,
    ):
        print("\n--- EVENT RECEIVED ---")

        # Print raw event object
        print("Event Object:", event)

        # Print any content parts
        if hasattr(event, "content") and event.content:
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    print("📝 Text:", part.text)
                elif hasattr(part, "function_call") and part.function_call:
                    print("🛠 Function Call:", part.function_call)
                else:
                    print("🔧 Other Part:", part)

        # Capture final output when done
        if event.is_final_response() and event.content and event.content.parts:
            final_text = event.content.parts[0].text

    print("\n💾 Saving session to long-term memory...")
    session = await runner.session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )
    await memory_service.add_session_to_memory(session)
    print("📚 Session saved.")

    print("\n🎉 FINAL AGENT RESPONSE:\n")
    print(final_text)


print("✔ Runner ready. Call run_once() to begin!")


Setting up runner and execution flow...
✔ Runner ready. Call run_once() to begin!


In [95]:
await run_once("I want to understand binary search from scratch.")



🚀 Starting new learning session for: 'I want to understand binary search from scratch.'
🗑 Previous session deleted.
✨ New session created.
✔ Runner initialized
🤖 Running agent interaction...



--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'binary search from scratch'
        },
        id='adk-83f89ea7-a1b4-4048-b3a9-035661ad70b1',
        name='planner_agent'
      ),
      thought_signature=b"\n\xe4\x01\x01\xd1\xed\x8ao\x86\xe0\xeb\xd7J\xa3\xf3\x0b\xbc\x1a'\xd5\x14 V\xbb\xe1\xaf\x0e.\xb0\xd1\xaa\xc0\r(\xac\x15B\xa1\xb19\xcc\xd1^\xe9\xb9\x9b\x80RP\xa9>\xa0\x8b_\xf2v\x95\xb0L\x81\xb9\xf0\x13\x88*\xf0\x14\x04p\xdf\x19\xf3\x1b\x0e\xc3>\x8a\xa7\xc7\xad\xb2\xec2y\x9dy5U)\xab\xa0Y\n\t\xee[=...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=18,
  prompt_token_count=350,
  prompt_tokens_details=[
    ModalityTokenCount(
    


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'Explain the concept and prerequisites of binary search, including what it is, its efficiency, and the critical condition (sorted data) required for it to work.'
        },
        id='adk-759f0875-6e5b-47bd-bae7-5a4649100444',
        name='tutor_agent'
      ),
      thought_signature=b'\n\xff\x02\x01\xd1\xed\x8aod$\xa4ir\xe3\xe2\xde1S\xef\xaa\x03\xfaw\x85dT\xb9\xaa\x92.\xed\x07F\xfb\xa4\xad[\xb1"\xe8\xb8\x0e\x7f\x98;^L2PY\xec\x95\xf2A\x06\x03^\x80F\xc7t\xebJZ=\xe6\x9e\x00\xf6%C\xe6UW\xceq`_\x87\x1e7\xf8\xef\x00\x93\xac\x00D\x9d\x9c\xd3\xbb\xa3\xc6\x19\xd1\x80...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetada


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'result': 'completed',
          'step_name': 'Step 1: Understand the Concept & Prerequisites',
          'topic': 'binary search'
        },
        id='adk-5925b9db-1495-4863-a23c-8943c1b660cf',
        name='update_progress'
      ),
      thought_signature=b'\n\xdd\x01\x01\xd1\xed\x8ao\x82)@x\xce\xf8\xe18\xcb\xbc!)^Ayy\xf5\xd2or\xf2\xaa\x1c\x92bK\x92\x84\x92\xbbW-\xf1\x83\xd2\x1e\x05\x84\xf0\x1a\x1e\xb3u!G\x05\xab\x9f\xb6\x91\x9c\xd50\x8b\x02\x88Yr\x8a\xf2\xf7&n\xb8\xe5\xc7\x0f\xaeL\xb9\ns\xa0Z[\xc38\xb0\x1f+\xa9\xf1\x13\xd1{\xb3C\xba\x8e...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=36


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'Create a short quiz on the concept and prerequisites of binary search, including what it is, its efficiency, and the critical condition (sorted data) required for it to work. Provide multiple choice questions.'
        },
        id='adk-a960d9f3-c10e-4138-a52a-29bc775635ab',
        name='quiz_agent'
      ),
      thought_signature=b"\n\xc6\x01\x01\xd1\xed\x8aok_$\xdc\xb9\xe2\x83\xabmTx\xd6\x9b\xd6\xe2,\xadF\xa4\x86\xfaw^\xe0P<`\xc4\x17\x07\xca\xe8\xfa)\xa9\xd3\x1f\x95\x1fN\x1drf\x7f\xd4\xec\x8b\xbeo\xa6\xe4\xe0\xe8n@\x88\x81z\xc2\xdc'\x10\x1e\x94l\xd4/\xab\x10\xd9\x17\x8e\xee\xa0\x9c7\x17B\xbe`vZ\xa3\x04}\x969(\xb4...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metad


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""I've prepared a short quiz for you to check your understanding of the concept and prerequisites of binary search. Please provide your answers to the following questions:

**Question 1:** What is the primary prerequisite for successfully applying a binary search algorithm, and how does it fundamentally operate?
a) The data must be stored in a hash map; it checks each element sequentially.
b) The data must be sorted; it repeatedly divides the search interval in half.
c) The data must be unsorted; it compares the target with the middle element.
d) The data must be stored in a linked list; it performs a breadth-first search.

**Question 2:** What is the worst-case time complexity of binary search for an array of `n` elements?
a) O(n)
b) O(n log n)
c) O(log n)
d) O(1)

Please provide your answers!""",
      thought_signature=b'\n\xd9\x02\x01\xd1\xed\x8ao\xa5\xe9\xef\xf9\

In [96]:
await run_once("teach me about addition")




🚀 Starting new learning session for: 'teach me about addition'
🗑 Previous session deleted.
✨ New session created.
✔ Runner initialized
🤖 Running agent interaction...



--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'addition'
        },
        id='adk-18863be6-18a5-4400-90e2-5935ca18c52c',
        name='planner_agent'
      ),
      thought_signature=b'\n\xb2\x02\x01\xd1\xed\x8aoR\x89>3\xca\x94\x01\xa35_\xe9\x88\x14\xd2\xfb2\x84\xbe}\x16\xd6\xfd\x88\xa9J\x86C\xb8}\xe2\xdbRZg\xf1\xf0d\x08TGC\xf4_\xd0TO\xe1\xc9F\x10Y\xc9\x9d\x84\xbc\xaf8Oi\xc0p\x1c\x1e\xac(A?]\xce\xacp\xea\xb8\x98\x0ef\xf7}\xd8\x1e\xd5\x85\xaf\x81\x1d>P\x8fZ...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=15,
  prompt_token_count=345,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'Explain what addition means as combining quantities to find a total.'
        },
        id='adk-6d5d6e28-4794-450e-842b-52f24e7f4838',
        name='tutor_agent'
      ),
      thought_signature=b'\n\xea\x02\x01\xd1\xed\x8ao\xaf\x08\xb4\xa4\xcfm%\xbb\xaf,\x12q\x16\xc4\xee\xfcu\xe7\xa1\xc6\x9b\xd6\x11\xffL\xe9\x0c\xe6\xfb\x85l\xfd1\x92\x91ft\xa9\x1dJ\xeenJ\xaa4\xf3\x81\x9d\xd6\x8d\xd8\xf6\xd8\x1dg\xc5\xe05\xae\xfc\xe2^w\xadN\rj\x10\x90|\xdc\x04\x1f\x19\x8f\x0b\xd4Si\xef]\x94r\xe8\r\xb9\xc2\x95\xc9...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=26,
  prompt_token_count=489,
  prom


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'result': 'Explained',
          'step_name': 'Understanding the Concept',
          'topic': 'addition'
        },
        id='adk-84c2f611-c9b3-43aa-918d-f3116f22ed18',
        name='update_progress'
      ),
      thought_signature=b'\n\xe3\x01\x01\xd1\xed\x8ao\xa0\xcc\xd1\xe5\xcd\x1fAd\xd2\xc2\xa6\x80U\xac\xe7:\x7f\x943\x8a$P\x82\xea+\xd9^~o\xafW={\xd3k\xcfTt\x1f\x9e~\xd6\xe7\xf8\x86\x12\xc3\xac\x91\xa1\x8a2\x17\xd4\x94\x16v\xd0\xcf\xaf\xf9\xc7\xe9\x0c\x96\xed\x8f\x1bk\x12\xdf\x9e\x89\xd7-$\x80\xecx\xed\xb1T\xeb`TV\xc2\x8f\x84...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=29,
  prompt_t


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'Quiz me on what addition means as combining quantities to find a total.'
        },
        id='adk-4b9dcf8b-1492-411b-942f-e85d55b951ed',
        name='quiz_agent'
      ),
      thought_signature=b"\n\x81\x03\x01\xd1\xed\x8ao\xe3\xaa\xce2\xcda\x05-\x0b\x96\xb9\xf5)\r`\x9a[\xdb\xfb\xd0\x9e\xff\xb5\xa6\xb6q\xdb\x92\x95\xcc\xa3\x14Y\xe9*\xb6\x17\xd1G\xc2$Pr\x07\xf8\xac\xad\xae@3\x98\xd6\x01\xea=\x9a\xa7b\x01\\/\xf4'\xfb\x01\xd0znk\x90YJ\x1b\xdf\xf6Zzu\xb1#'F\x8d\xa5llI\xa0\xb6...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  cache_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT


--- EVENT RECEIVED ---
Event Object: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Please answer the following questions:

1.  In simple terms, when you add, what are you doing with different quantities or groups of items?
2.  What is the final number called that represents all the items once you've brought them together?""",
      thought_signature=b'\n\xce\x02\x01\xd1\xed\x8ao%\xb6I.\x0b9\x0f\x85\xae\x14yse\xd8\x12\x83a\x9f;\x83\xac\x8b\x9aC\xef\x8a\xa6\xe4\xb7\xf2\x99\x1f\xe2\xe6\x19\xd1\xec\xb9\xe3c\xa2\xef\x06\xd3X\x11\xfe\x1a\x8a\xc38\x84\xfaZ)\x85\xc8\xbc7\xccH\xe2_\xbal\xbc\x8f/\xb0av\x9b\xa27\xa2c3TT\xb1\xe6\xdf\xd6\xba\x8e_\x93\x8d\xe7...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  cache_tokens_details=[
    ModalityTokenCount(
 